In [9]:
import pandas as pd
import joblib
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# =================================================================
# FASE 1: DESENVOLVIMENTO DO MODELO (Ajustado para Performance)
# =================================================================

# Lendo o dataset
df = pd.read_csv('buscape.csv')
df = df[['review_text', 'polarity']].dropna()
df['review_text'] = df['review_text'].str.lower().str.strip()

X = df['review_text']
y = df['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AJUSTES NO PIPELINE:
# 1. ngram_range=(1, 2): Analisa palavras sozinhas e em duplas (ex: "não" + "bom").
# 2. class_weight='balanced': Dá mais peso para a classe minoritária (Negativos),
#    corrigindo o erro de classificar reclamações como positivas.
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(class_weight='balanced', solver='liblinear'))
])

print("Treinando o modelo ajustado... Aguarde.")
pipeline.fit(X_train, y_train)

# Métricas
y_pred = pipeline.predict(X_test)
print("\n--- Novas Métricas de Desempenho ---")
print(f"Acurácia Geral: {accuracy_score(y_test, y_pred):.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Serialização
model_filename = "modelo_sentimento_pipeline.joblib"
joblib.dump(pipeline, model_filename)
print(f"\nModelo serializado: {model_filename}")

# =================================================================
# FASE 2: TESTE DE INTEGRAÇÃO (Simulação de Uso Real)
# =================================================================

print("\n" + "="*50)
print("TESTE COM O MODELO RECALIBRADO")
print("="*50)

modelo_para_uso = joblib.load(model_filename)

try:
    with open('data.json', 'r', encoding='utf-8') as f:
        dados_novos = json.load(f)

    for i, item in enumerate(dados_novos):
        texto_input = item['text']
        classe_predita = modelo_para_uso.predict([texto_input])[0]
        probabilidade = modelo_para_uso.predict_proba([texto_input]).max()
        sentimento = "Positivo" if classe_predita == 1.0 else "Negativo"

        print(f"Item {i+1}:")
        print(f"Texto: {texto_input[:60]}...")
        print(f"Resultado: {sentimento} (Confiança: {probabilidade:.2f})")
        print("-" * 30)

except FileNotFoundError:
    print("Erro: Arquivo 'data.json' não encontrado.")

Treinando o modelo ajustado... Aguarde.

--- Novas Métricas de Desempenho ---
Acurácia Geral: 0.90

Relatório de Classificação:
              precision    recall  f1-score   support

         0.0       0.48      0.71      0.57       342
         1.0       0.97      0.92      0.94      3355

    accuracy                           0.90      3697
   macro avg       0.72      0.81      0.76      3697
weighted avg       0.92      0.90      0.91      3697


Modelo serializado: modelo_sentimento_pipeline.joblib

TESTE COM O MODELO RECALIBRADO
Item 1:
Texto: O atendimento foi excelente e a entrega chegou super rápida,...
Resultado: Positivo (Confiança: 0.96)
------------------------------
Item 2:
Texto: O produto veio com defeito e a troca está demorando muito. P...
Resultado: Negativo (Confiança: 0.96)
------------------------------
Item 3:
Texto: Chegou dentro do prazo, mas o manual de instruções é confuso...
Resultado: Positivo (Confiança: 0.78)
------------------------------
Item 4:
Texto:

In [8]:
# Teste manual rápido

# Mapeamento: O dataset tem 0.0 (Negativo) e 1.0 (Positivo)
def get_label(polarity):
    # Usamos float() para garantir a comparação correta se o retorno for numérico
    if float(polarity) == 1.0:
        return "Positivo"
    else:
        return "Negativo"

exemplos = [
    "O produto é merda, chegou com defeito!",
    "Péssimo, veio quebrado e o atendimento não responde.",
    "É razoável, funciona mas poderia ser mais barato."
]

preds = pipeline.predict(exemplos)
probs = pipeline.predict_proba(exemplos)

print("\n--- Teste Prático ---")
for text, pred, prob in zip(exemplos, preds, probs):
    # Transformando o valor numérico (0.0/1.0) em texto usando sua função
    label_texto = get_label(pred)

    # A probabilidade retorna uma lista: [prob_negativo, prob_positivo]
    # Pegamos a maior delas para mostrar a confiança da decisão tomada
    confianca = max(prob)

    print(f"Texto: '{text}'")
    print(f"-> Previsão: {label_texto} (Confiança: {confianca:.2f})")
    print("-" * 30)


--- Teste Prático ---
Texto: 'O produto é merda, chegou com defeito!'
-> Previsão: Negativo (Confiança: 0.74)
------------------------------
Texto: 'Péssimo, veio quebrado e o atendimento não responde.'
-> Previsão: Negativo (Confiança: 0.96)
------------------------------
Texto: 'É razoável, funciona mas poderia ser mais barato.'
-> Previsão: Positivo (Confiança: 0.59)
------------------------------
